In [5]:
pip install flask flask-cors ipywidgets pillow zipfile36

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from flask import Flask, request, send_file
from flask_cors import CORS
from PIL import Image, ImageEnhance, ImageOps
from io import BytesIO
import zipfile
import threading

app = Flask(__name__)
CORS(app)

UPLOAD_FOLDER = 'uploads'
AUGMENTED_FOLDER = 'augmented'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(AUGMENTED_FOLDER, exist_ok=True)

def augment_image(image_path, operations):
    img = Image.open(image_path)
    for operation in operations:
        if operation == 'rotate':
            img = img.rotate(90)
        elif operation == 'flip':
            img = ImageOps.flip(img)
        elif operation == 'mirror':
            img = ImageOps.mirror(img)
        elif operation == 'zoom':
            width, height = img.size
            img = img.crop((width*0.1, height*0.1, width*0.9, height*0.9))
            img = img.resize((width, height), Image.LANCZOS)
        elif operation == 'color':
            enhancer = ImageEnhance.Color(img)
            img = enhancer.enhance(2.0)
    return img

@app.route('/upload', methods=['POST'])
def upload_files():
    files = request.files.getlist('files')
    operations = request.form.getlist('operations')
    zip_buffer = BytesIO()

    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED) as zip_file:
        for file in files:
            filename = file.filename
            file_path = os.path.join(UPLOAD_FOLDER, filename)
            file.save(file_path)
            
            augmented_img = augment_image(file_path, operations)
            augmented_img_path = os.path.join(AUGMENTED_FOLDER, filename)
            augmented_img.save(augmented_img_path)
            
            zip_file.write(augmented_img_path, os.path.basename(augmented_img_path))

    zip_buffer.seek(0)
    return send_file(zip_buffer, as_attachment=True, attachment_filename='augmented_images.zip')

def run_flask():
    app.run(debug=True, use_reloader=False)

# Run the Flask app in a separate thread
threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests

# File upload widget
upload_widget = widgets.FileUpload(accept='image/*', multiple=True)

# Checkboxes for augmentation operations
rotate_checkbox = widgets.Checkbox(value=False, description='Rotate')
flip_checkbox = widgets.Checkbox(value=False, description='Flip')
mirror_checkbox = widgets.Checkbox(value=False, description='Mirror')
zoom_checkbox = widgets.Checkbox(value=False, description='Zoom')
color_checkbox = widgets.Checkbox(value=False, description='Color Adjustment')

# Button to trigger the upload and augmentation process
upload_button = widgets.Button(description='Upload and Augment')

# Output widget to display the download link
output_widget = widgets.Output()

def on_button_click(b):
    with output_widget:
        clear_output()
        files = upload_widget.value
        operations = []
        if rotate_checkbox.value:
            operations.append('rotate')
        if flip_checkbox.value:
            operations.append('flip')
        if mirror_checkbox.value:
            operations.append('mirror')
        if zoom_checkbox.value:
            operations.append('zoom')
        if color_checkbox.value:
            operations.append('color')

        if files:
            form_data = {
                'files': (f['metadata']['name'], f['content']) for f in files.values()
            }
            for operation in operations:
                form_data[operation] = (None, operation)
            
            response = requests.post('http://localhost:5000/upload', files=form_data)
            
            if response.status_code == 200:
                download_link = widgets.HTML(
                    value=f"<a href='data:application/zip;base64,{response.content.decode('latin1')}' download='augmented_images.zip'>Download Augmented Images</a>"
                )
                display(download_link)
            else:
                display(widgets.HTML(value=f"<p>Upload failed with status {response.status_code}</p>"))

upload_button.on_click(on_button_click)

# Display the UI components
display(upload_widget)
display(rotate_checkbox, flip_checkbox, mirror_checkbox, zoom_checkbox, color_checkbox)
display(upload_button)
display(output_widget)


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

Checkbox(value=False, description='Rotate')

Checkbox(value=False, description='Flip')

Checkbox(value=False, description='Mirror')

Checkbox(value=False, description='Zoom')

Checkbox(value=False, description='Color Adjustment')

Button(description='Upload and Augment', style=ButtonStyle())

Output()